In [ ]:
import pandas as pd
import numpy as np
import pickle
import time
import schedule
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
def job():
    tem = time.time()
    df = pd.read_csv('./main.csv')
    df['Time'] = pd.to_datetime(df['Time'])
    df.drop(df[df.isnull().sum(axis = 1) > 3].index, axis = 0, inplace = True)
    df.drop(df[df['HG_ARRAY_02'].isnull()].index, axis = 0, inplace = True)
    df.drop(df[df['HG_ARRAY_02'] < 500].index, axis = 0, inplace = True)
    df.drop(df[df['Wind_Velocity'] > 10].index, axis = 0, inplace = True)
    df.drop([17], axis = 0, inplace = True)
    name = 'solarDataProject'
    run = 1
    while run == 1:
        for index, row in df[df['Wind_Velocity'].isnull()].iterrows():
            df.loc[index, 'Wind_Velocity'] = np.random.normal(df['Wind_Velocity'].mean(), df['Wind_Velocity'].std())
        for index, row in df[df['TT_16'].isnull()].iterrows():
            df.loc[index, 'TT_16'] = df.loc[index, 'TT_11'] + np.random.normal((df['TT_16'] - df['TT_11']).mean(), (df['TT_16'] - df['TT_11']).std())
        for index, row in df[df['PYNM_02'].isnull()].iterrows():
            df.loc[index, 'PYNM_02'] = np.random.normal(df['PYNM_02'].mean(), df['PYNM_02'].std())
        traindata, testdata, trainanswers, testanswers = train_test_split(
            df.iloc[:, 2:7], df.iloc[:, 7], test_size = 0.2)
        n1, n2 = np.random.randint(3, 21), np.random.randint(3, 21)
        lr = 'constant' if np.random.randint(0, 2) == 0 else 'adaptive'
        maxiter, iternochange = np.random.randint(5000, 15000), np.random.randint(0, 2500)
        model = MLPRegressor(hidden_layer_sizes = (n1, n2,),
                             learning_rate = lr,
                             max_iter = maxiter,
                             verbose = False,
                             early_stopping = True,
                             validation_fraction = 0.4,
                             n_iter_no_change = iternochange)
        st = time.time()
        model.fit(traindata, trainanswers)
        et = time.time()
        trainpredictions = model.predict(traindata)
        trainr2 = r2_score(trainanswers, trainpredictions)
        testpredictions = model.predict(testdata)
        testr2 = r2_score(testanswers, testpredictions)
        try:
            metrics = pd.read_csv('./metrics.csv')
            version = metrics.loc[metrics.shape[0] - 1, 'Version'].split('.')
            if testr2 >= metrics['testr2'].max():
                pickle.dump(model, open('./bestmodel.nnm', 'wb'))
                version[1] = str(int(version[1]) + 1)
                version[2] = '0'
                run = 0
            else:
                version[2] = str(int(version[2]) + 1)
            version = version[0] + '.' + version[1] + '.' + version[2]
            metrics = metrics.append(pd.DataFrame([[name, version, n1, n2, lr, trainr2, testr2, et - st, model.n_iter_, maxiter, iternochange, df.shape[0]]],
                                                  columns = ['Name', 'Version', 'HL1', 'HL2', 'learning', 'trainr2', 'testr2', 'time', 'iterations', 'max_iter', 'n_iter_no_change', 'datasetsize']),
                                     sort = False)
            pickle.dump(model, open('./models/' + name + '.' + version + '.nnm', 'wb'))
        except:
            version = '0.0.0'
            pickle.dump(model, open('./bestmodel.nnm', 'wb'))
            metrics = pd.DataFrame([[name, str(version), n1, n2, lr, trainr2, testr2, et - st, model.n_iter_, maxiter, iternochange, df.shape[0]]],
                                                  columns = ['Name', 'Version', 'HL1', 'HL2', 'learning', 'trainr2', 'testr2', 'time', 'iterations', 'max_iter', 'n_iter_no_change', 'datasetsize'])
            pickle.dump(model, open('./models/' + name + '.' + version + '.nnm', 'wb'))
            run = 0
        metrics.to_csv(r'metrics.csv', index = False)

print(schedule.every(5).seconds.do(job))
while True:
    schedule.run_pending()
    time.sleep(1)